In [69]:
import pandas as pd
import requests
import sys
import numpy as np

In [ ]:
tfs = pd.read_csv("original_data/lambert_supp_tables.csv")
tfs = tfs[tfs["Is TF?"] == "Yes"][["EnsemblId", "symbol"]]
tfs.to_csv("lambert_tfs.csv")
tfs.head()

,EnsemblId,symbol
1,ENSG00000137203,TFAP2A
2,ENSG00000008196,TFAP2B
3,ENSG00000087510,TFAP2C
4,ENSG00000008197,TFAP2D
5,ENSG00000116819,TFAP2E


In [ ]:
ads = pd.read_csv("original_data/known_ADs_considering_isoforms_and_canonical.csv")
ads.head()

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
0,ABRAXAS1,121,200,Q6UWZ7,[['Q6UWZ7']],ENST00000321945,Q6UWZ7,DelRosso et al.,CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,NaN
1,AEBP1,1088,1158,Q8IUX7,[['Q8IUX7']],NaN,Q8IUX7,Staller Activity Data,TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,NaN
2,AHCTF1,1445,1698,Q8WYP5,"[['Q8WYP5'], ['Q8WYP5']]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto / transcriptionalactivity...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,NaN
3,AHR,118,126,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,LLQALNGFV,NaN
4,AHR,266,268,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,FAI,NaN


In [ ]:
rds = pd.read_csv("original_data/tycko_soto_delrosso_RD_coordinate_data.csv")
rds.head()

,UniprotId,DBDs
0,A0A024R0Y4,"[(11, 90)]"
1,A0A087X169,"[(21, 140)]"
2,A0A0A0MRP6,"[(11, 140), (561, 650), (881, 960)]"
3,A0A0B4J2G0,"[(2, 81), (352, 431), (612, 731)]"
4,A0A0C4DFL8,"[(581, 700), (831, 910), (931, 1050)]"


In [ ]:
tfsmapped = pd.read_csv("maps/ID Mapping Dec 10 2024.tsv", delimiter="\t")
tfsmapped.head()

,From,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Tissue specificity,Motif,Protein families,Compositional bias,Sequence similarities,Bgee,ExpressionAtlas
0,ENSG00000137203,P05549,reviewed,AP2A_HUMAN,Transcription factor AP-2-alpha (AP2-alpha) (A...,TFAP2A AP2TF TFAP2,Homo sapiens (Human),437,NaN,"MOTIF 57..62; /note=""PPxY motif""",AP-2 family,"COMPBIAS 23..48; /note=""Polar residues""; /evid...",SIMILARITY: Belongs to the AP-2 family. {ECO:0...,ENSG00000137203; Expressed in upper leg skin a...,P05549; baseline and differential.;
1,ENSG00000137203,A0A6E1XE14,unreviewed,A0A6E1XE14_HUMAN,Transcription factor AP-2-alpha (AP-2 transcri...,TFAP2A hCG_37170,Homo sapiens (Human),439,NaN,NaN,AP-2 family,"COMPBIAS 25..50; /note=""Polar residues""; /evid...",SIMILARITY: Belongs to the AP-2 family. {ECO:0...,ENSG00000137203; Expressed in upper leg skin a...,NaN
2,ENSG00000137203,C1K3N0,unreviewed,C1K3N0_HUMAN,Transcription factor AP-2-alpha (AP-2 transcri...,TFAP2A,Homo sapiens (Human),328,NaN,NaN,AP-2 family,"COMPBIAS 23..48; /note=""Polar residues""; /evid...",SIMILARITY: Belongs to the AP-2 family. {ECO:0...,ENSG00000137203; Expressed in upper leg skin a...,NaN
3,ENSG00000137203,C9J6N8,unreviewed,C9J6N8_HUMAN,Transcription factor AP-2 alpha,TFAP2A,Homo sapiens (Human),173,NaN,NaN,NaN,"COMPBIAS 7..22; /note=""Polar residues""; /evide...",NaN,ENSG00000137203; Expressed in upper leg skin a...,C9J6N8; baseline and differential.;
4,ENSG00000137203,C9JXZ2,unreviewed,C9JXZ2_HUMAN,Transcription factor AP-2 alpha,TFAP2A,Homo sapiens (Human),136,NaN,NaN,AP-2 family,NaN,SIMILARITY: Belongs to the AP-2 family. {ECO:0...,ENSG00000137203; Expressed in upper leg skin a...,C9JXZ2; baseline and differential.;


In [33]:
def ad_rd_annotate(uniprot):
    numads = len(ads[ads["uniprotID"] == uniprot])
    numrds = len(rds[rds["UniprotId"] == uniprot])
    if numads + numrds == 0:
        classification = "none"
    elif (numads > 0) and (numrds > 0):
        classification = "both"
    elif numads > 0:
        classification = "AD only"
    elif numrds > 0:
        classification = "RD only"
    return numads, numrds, classification

numadslist = []
numrdslist = []
classificationlist = []
for uniprot in tfsmapped["Entry"]:
    numads, numrds, classification = ad_rd_annotate(uniprot)
    numadslist.append(numads)
    numrdslist.append(numrds)
    classificationlist.append(classification)

tfsmapped["num ADs"] = numadslist
tfsmapped["num RDs"] = numrdslist
tfsmapped["classification"] = classificationlist

tfsmapped.head()

,From,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Tissue specificity,Motif,Protein families,Compositional bias,Sequence similarities,Bgee,ExpressionAtlas,num ADs,num RDs,classification
0,ENSG00000137203,P05549,reviewed,AP2A_HUMAN,Transcription factor AP-2-alpha (AP2-alpha) (A...,TFAP2A AP2TF TFAP2,Homo sapiens (Human),437,NaN,"MOTIF 57..62; /note=""PPxY motif""",AP-2 family,"COMPBIAS 23..48; /note=""Polar residues""; /evid...",SIMILARITY: Belongs to the AP-2 family. {ECO:0...,ENSG00000137203; Expressed in upper leg skin a...,P05549; baseline and differential.;,1,0,AD only
1,ENSG00000137203,A0A6E1XE14,unreviewed,A0A6E1XE14_HUMAN,Transcription factor AP-2-alpha (AP-2 transcri...,TFAP2A hCG_37170,Homo sapiens (Human),439,NaN,NaN,AP-2 family,"COMPBIAS 25..50; /note=""Polar residues""; /evid...",SIMILARITY: Belongs to the AP-2 family. {ECO:0...,ENSG00000137203; Expressed in upper leg skin a...,NaN,0,0,none
2,ENSG00000137203,C1K3N0,unreviewed,C1K3N0_HUMAN,Transcription factor AP-2-alpha (AP-2 transcri...,TFAP2A,Homo sapiens (Human),328,NaN,NaN,AP-2 family,"COMPBIAS 23..48; /note=""Polar residues""; /evid...",SIMILARITY: Belongs to the AP-2 family. {ECO:0...,ENSG00000137203; Expressed in upper leg skin a...,NaN,0,0,none
3,ENSG00000137203,C9J6N8,unreviewed,C9J6N8_HUMAN,Transcription factor AP-2 alpha,TFAP2A,Homo sapiens (Human),173,NaN,NaN,NaN,"COMPBIAS 7..22; /note=""Polar residues""; /evide...",NaN,ENSG00000137203; Expressed in upper leg skin a...,C9J6N8; baseline and differential.;,0,0,none
4,ENSG00000137203,C9JXZ2,unreviewed,C9JXZ2_HUMAN,Transcription factor AP-2 alpha,TFAP2A,Homo sapiens (Human),136,NaN,NaN,AP-2 family,NaN,SIMILARITY: Belongs to the AP-2 family. {ECO:0...,ENSG00000137203; Expressed in upper leg skin a...,C9JXZ2; baseline and differential.;,0,0,none


In [35]:
tfsmapped["classification"].value_counts()

classification
none       5010
RD only     701
both        369
AD only      90
Name: count, dtype: int64

In [ ]:
%%skip
def gtex_getgenecode(ensemblid):
    server = "https://gtexportal.org/api/v2/reference/gene?"
    ext = "geneId=" + ensemblid
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    
    decoded = r.json()

    return decoded["data"][0]["gencodeId"]

tfsmapped["gencodeId"] = tfsmapped["From"].apply(gtex_getgenecode)
tfsmapped.to_csv("tfsmapped-gencodes.csv")

In [ ]:
%%skip
def gtex_fullexpression(ensemblid):
    server = "https://gtexportal.org/api/v2/expression/geneExpression?"
    ext = "gencodeId=" + ensemblid
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    
    decoded = r.json()

    return pd.DataFrame(decoded["data"])


def gtex_fullexpression_clean(ensemblid, save_csv, dirstring):
    """
    Ensemblid: 
    Save_csv (boolean): 
    dirstring (string): directory to save the csv into
    value (string): 1D df of mean, median, std, count from the expression results to return
    """
    fulldf = gtex_fullexpression(ensemblid)

    newdf = fulldf[["tissueSiteDetailId"]].set_index("tissueSiteDetailId")
    newdf["mean"] = fulldf["data"].apply(np.mean).to_list()
    newdf["median"] = fulldf["data"].apply(np.mean).to_list()
    newdf["std"] = fulldf["data"].apply(np.std).to_list()
    newdf["count"] = fulldf["data"].apply(len).to_list()
    
    def select_df(label):
        return newdf[[label]].rename({label: ensemblid}, axis=1)

    if save_csv:
        newdf.to_csv(dirstring + ensemblid + "_expression.csv")

    return select_df("mean"), select_df("median"), select_df("std"), select_df("count")


################## Run on TFs

means, medians, stds, counts = pd.DataFrame({}), pd.DataFrame({}), pd.DataFrame({}), pd.DataFrame({})

for ensemblId in tfsmapped["gencodeId"].unique():
    try:
        mean_i, median_i, std_i, count_i = gtex_fullexpression_clean(ensemblId, False, "TF_expression/")
        means = means.join(mean_i, how = "outer")
        medians = medians.join(median_i, how = "outer")
        stds = stds.join(std_i, how = "outer")
        counts = counts.join(count_i, how = "outer")
    except:
        print(ensemblId + "error")

means.to_csv("ensemblId_mean.csv")
medians.to_csv("ensemblId_median.csv")
stds.to_csv("ensemblId_stds.csv")
counts.to_csv("ensemblId_counts.csv")

ENSG00000267281.2error
ENSG00000267179.1error
ENSG00000264668.1error
ENSG00000100146.16error
ENSG00000064489.22error
